# Peewee

Peewee is a lightweight Python ORM (Object Relational Mapper) that allows you to map objects in your Python code to rows in a relational database.
Peewee provides a simple, expressive API for working with databases, and allows you to easily and efficiently interact with your data without having to write complex SQL queries.

In [21]:
import os
from datetime import datetime
from typing import Type

from faker import Faker
from faker.providers.date_time import Provider
from peewee import DateTimeField, SqliteDatabase, Model, CharField, IntegerField, FloatField, AutoField, DateField,\
    ModelSelect

from playhouse.migrate import migrate, SchemaMigrator
from playhouse.sqlite_ext import JSONField

from Util import now

In [22]:
# SQLite
DATABASE_NAME = "sqlite.db"
db: SqliteDatabase = SqliteDatabase(DATABASE_NAME)
db.connect()  # This creates the db if not existing
fake = Faker()

In [ ]:
# MySQL
# pip install pymysql
# pip install cryptography
# DATABASE_NAME = "mysql.db"
# db: SqliteDatabase = MySQLDatabase('my_app', user='app', password='db_password', host='10.1.0.8', port=3306)

In [23]:
def run_cmd(cmd):
    os.system('"' + cmd + '"')

# noinspection PyProtectedMember
def get_columns_name_model(model):
    return model._meta.table_name


# noinspection PyProtectedMember
def get_database(model):
    return model._meta.database


def get_columns_name_db(model):
    return [tuples[1] for tuples in
            get_database(model).cursor().execute("PRAGMA table_info({})".format(get_columns_name_model(model)))]


def get_fields_name_fields_value(model) -> dict:
    return model.__dict__["__data__"]


# noinspection PyProtectedMember
def print_model_infos(model):
    database = model._meta.database
    table_name = model._meta.table_name
    primary_key = model._meta.primary_key
    fields = model._meta.fields
    pr(database, table_name, primary_key, fields)


def add_missing_columns_to_db(model, columns, columns_type=[str | list], debug=True):
    """ Update the Model code after adding columns """
    database = get_database(model)
    db_columns = get_columns_name_model(model)
    column_name = get_columns_name_model(model)
    migrator = SchemaMigrator(database)
    for i in range(len(columns)):
        column = columns[i].replace("-", "_")
        column_type = columns_type[i] if type(columns_type) is list else columns_type
        if column in db_columns:
            continue
        migrate(migrator.add_column(column_name, column, type_to_field(column_type)))
    if debug:
        run_cmd("python -m pwiz -e sqlite {}".format(get_database(model)))


def fill_rows(model, columns_order: list[str], values: list[list[object]], debug=True):
    try:
        values[0][0]
    except IndexError:
        values = [values]
    db_columns = get_columns_name_db(model)
    indexes = [columns_order.index(index) for index in set(columns_order) - set(db_columns)]
    columns_order = [column for column in columns_order if column in db_columns]
    rows = [dict(zip(columns_order, [value[i] for i in range(len(value)) if i not in indexes]))
            for value in values]
    q = model.insert_many(rows)
    q.execute()
    if debug:
        print(now(), q.sql())


def default_naming_convention_table(model: Model):
    return model.__name__.upper()


def type_to_field(val: object):
    python_type = type(val)
    if python_type is str and str(val).isdecimal():
        python_type = float
    if python_type is str:
        return CharField(null=True)
    elif python_type is int:
        return IntegerField(null=True)
    elif python_type is float:
        return FloatField(null=True)
    elif python_type is datetime:
        return DateTimeField(null=True)
    elif python_type in [list, tuple, dict]:
        return JSONField(json_dumps=val, null=True)
    else:
        raise TypeError(str(python_type) + " is not defined")

In [24]:
all_countries: list[dict[str, str]] = Provider.countries
all_countries[0]

Country(name='Andorra', timezones=['Europe/Andorra'], alpha_2_code='AD', alpha_3_code='AND', continent='Europe', capital='Andorra la Vella')

## Vocabulary

Model class corresponds to a Database table
Field instance corresponds to a Column on a table
Model instance corresponds to a Row in a database table

## Create named model tables linked to a db

### Field types table
https://docs.peewee-orm.com/en/latest/peewee/models.html#field-types-table

### Field initialization arguments
https://docs.peewee-orm.com/en/latest/peewee/models.html#field-initialization-arguments

In [25]:
from Util import pr


class BaseModel(Model):
    """ DRY model class """

    class Meta:
        database = db


# "person" model table created with the default naming convention
class Person(BaseModel):
    person_id = AutoField(primary_key=True)  # will be auto-incrementing & PK.
    name = CharField()
    birthday = DateField()
    register_time = DateTimeField(
        default=now)  # now will be call when an object is created, be careful with mutable argument


# "PET" model table created with raw naming
class Pet(BaseModel):
    # owner = ForeignKeyField(Person, backref='pets')
    owner = CharField()
    name = CharField()
    animal_type = CharField()

    class Meta:
        table_name = 'Pet'


# "Country" model table created with a function used for the naming
class Country(BaseModel):
    """ Because none of the fields are initialized with primary_key=True, an auto-incrementing primary key will automatically be created and named "id" """
    name = CharField(primary_key=True)
    alpha_2_code = CharField(column_name='alpha_2_code', null=True)
    alpha_3_code = CharField(column_name='alpha_3_code', null=True)
    capital = CharField(null=True)
    continent = CharField(null=True)
    timezones = JSONField(null=True)

    class Meta:
        table_function = default_naming_convention_table


_: Type[Country] = Country
Country, Country.name, Country.alpha_3_code

(<Model: Country>,
 <CharField: Country.name>,
 <CharField: Country.alpha_3_code>)

## Create/drop tables

In [26]:
db.drop_tables([Person, Pet, Country])
db.create_tables([Person, Pet, Country])

## Add tuple

In [27]:
person = Person(name=fake.name(), birthday=fake.date_time())
person.save()
q = Pet.insert(owner=fake.name(), name=fake.name(), animal_type=fake.name())
q.sql()

('INSERT INTO "Pet" ("owner", "name", "animal_type") VALUES (?, ?, ?)',
 ['Nathan Garcia', 'John Smith', 'Jessica Schwartz'])

## Add tuples

In [54]:
all_countries: list[dict[str, str]] = Provider.countries
db.drop_tables([Person, Pet, Country])
db.create_tables([Person, Pet, Country])
columns_order = list(map(lambda x: x.replace("-", "_"), all_countries[0].__dict__.keys()))
values = [list(country.__dict__.values()) for country in all_countries]
add_missing_columns_to_db(Country, columns_order, values[0])
fill_rows(Country, columns_order, values)


pydev debugger: Unable to find real location for: C:\Users\Alexis\AppData\Local\Temp\ipykernel_42628\999387368.py


IntegrityError: UNIQUE constraint failed: COUNTRY.name

# Queries

In [ ]:
query: ModelSelect = Country.select()
for country in query:
    _: Model = country
    print(country, country.alpha_3_code, country.capital)  # print(country) print the Country PK

In [ ]:
list(Country.select(Country.name))

In [ ]:
first_value, last_value_a, last_value_b = Country.select().first(), Country.select().order_by(
    Country.name.desc()).get(), list(Country.select(Country.name))[-1]
first_value, last_value_a, last_value_b

In [ ]:
Country.get_or_none(Country.name == "France"), Country.get_or_none(Country.name == "_France")

In [ ]:
Country.select().count(), Country.select().scalar()

In [ ]:
query: ModelSelect = query.where(Country.name.startswith("F"))
list(query), query.sql()

# Orders.select(fn.MAX(Orders.date))

# pwiz
The pwiz library is a Python library that helps automate the process of generating Python code from existing database schemas. This can be useful when working with large, complex databases and you want to quickly generate the necessary Python code for interacting with the database without having to manually write all the code yourself.

## Models generator
https://docs.peewee-orm.com/en/latest/peewee/playhouse.html#pwiz

"python -m pwiz -e sqlite db_file.db"
This command will generate a script that contains the necessary Python code for interacting with the database. The generated code will include classes that correspond to the tables in the database, and the fields of the classes will correspond to the columns in the tables.

# Sources
https://docs.peewee-orm.com/en/latest/